In [ ]:
# The rest of the code needs this package update

!pip install datasets==4.0.0

import datasets
print(datasets.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [ ]:
# Required installation

!pip install evaluate
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from tqdm import tqdm
import numpy as np
import pandas as pd
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from huggingface_hub import Repository
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import data to work with and the different tokenizers for teacher and student

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Postdoc/cleaned_stripped_mimic_notes.csv")
print(df.shape)

dataset = Dataset.from_pandas(df[["clean_relevant_note_truncate", "label"]])

# Load both tokenizers
teacher_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
student_tokenizer = AutoTokenizer.from_pretrained("gaunernst/bert-tiny-uncased")

# Custom collate function
def collate_fn(batch):
    texts = [x['clean_relevant_note_truncate'] for x in batch]
    labels = torch.tensor([x['label'] for x in batch])

    # Tokenize separately for teacher and student
    teacher_inputs = teacher_tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='pt')
    student_inputs = student_tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors='pt')

    return {
        'teacher_inputs': teacher_inputs,
        'student_inputs': student_inputs,
        'labels': labels
    }

(51695, 4)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [ ]:
# Prepare datasets that are not tokenized

# Split data into train and temp
temp_split = dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = temp_split["train"]
temp_dataset = temp_split["test"]

# Then split temp into val and test
val_test_split = temp_dataset.train_test_split(test_size=0.5, seed=42)
val_dataset = val_test_split["train"]
test_dataset = val_test_split["test"]

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)

In [ ]:
# Load fine-tuned teacher model
teacher_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Postdoc/Bio_ClinicalBERT-fine_tuned/v3")
teacher_model.eval()

# Load smaller student model
student_model = AutoModelForSequenceClassification.from_pretrained("gaunernst/bert-tiny-uncased", num_labels=2)
student_model.train()

model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at gaunernst/bert-tiny-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [ ]:
# Optimizer
optimizer = AdamW(student_model.parameters(), lr=3e-5)

# Distillation loss: cross-entropy with custom weights + KL divergence
weights = torch.tensor([0.72, 1.60]).to(device)
ce_loss_fn = nn.CrossEntropyLoss(weight=weights)
kl_loss_fn = nn.KLDivLoss(reduction="batchmean")
temperature = 2.0
alpha = 0.5  # balance between hard and soft loss


In [ ]:
# Function to evaluate student model on test set

def evaluate_model_on_test_set_raw(model, test_dataset, tokenizer_name="gaunernst/bert-tiny-uncased", batch_size=8):
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    # Create DataLoader that tokenizes on-the-fly
    def collate_fn(batch):
        texts = [x["clean_relevant_note_truncate"] for x in batch]
        labels = torch.tensor([x["label"] for x in batch])
        encodings = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
        return encodings, labels

    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # Move model to device
    model.eval()
    model.to(device)

    all_logits = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = batch
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            outputs = model(**inputs)
            logits = outputs.logits

            all_logits.append(logits.cpu())
            all_labels.append(labels.cpu())

    # Concatenate results
    logits = torch.cat(all_logits, dim=0)
    labels = torch.cat(all_labels, dim=0)
    probs = torch.nn.functional.softmax(logits, dim=1)
    preds = torch.argmax(probs, dim=1).numpy()

    # Compute metrics
    labels_np = labels.numpy()
    acc = accuracy_score(labels_np, preds)
    prec = precision_score(labels_np, preds)
    rec = recall_score(labels_np, preds)
    f1 = f1_score(labels_np, preds)
    auc = roc_auc_score(labels_np, probs[:, 1].numpy())
    cm = confusion_matrix(labels_np, preds)

    # Display results
    print("🔍 Test Set Evaluation:")
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall:    {rec:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print(f"AUROC:     {auc:.4f}")
    print("\nConfusion Matrix:")
    print(cm)

    print("\nDetailed Classification Report:")
    print(classification_report(labels_np, preds, digits=4))

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "auroc": auc,
        "confusion_matrix": cm
    }

# Metrics to compute

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1": f1.compute(predictions=preds, references=labels)["f1"],
        "recall": recall.compute(predictions=preds, references=labels)["recall"],
        "precision": precision.compute(predictions=preds, references=labels)["precision"]
    }

In [ ]:
# Evaluate student model before distillation

print("Untrained student model:")
results = evaluate_model_on_test_set_raw(student_model, test_dataset, tokenizer_name="gaunernst/bert-tiny-uncased")

Untrained student model:


Evaluating: 100%|██████████| 970/970 [00:15<00:00, 62.25it/s]

🔍 Test Set Evaluation:
Accuracy:  0.6919
Precision: 1.0000
Recall:    0.0004
F1 Score:  0.0008
AUROC:     0.5084

Confusion Matrix:
[[5365    0]
 [2389    1]]

Detailed Classification Report:
              precision    recall  f1-score   support

           0     0.6919    1.0000    0.8179      5365
           1     1.0000    0.0004    0.0008      2390

    accuracy                         0.6919      7755
   macro avg     0.8460    0.5002    0.4094      7755
weighted avg     0.7869    0.6919    0.5661      7755



In [ ]:
# Evaluate fine-tuned teacher model (sanity check)

print("Teacher model:")
results = evaluate_model_on_test_set_raw(teacher_model, test_dataset, tokenizer_name="emilyalsentzer/Bio_ClinicalBERT")

Teacher model:


Evaluating: 100%|██████████| 970/970 [03:50<00:00,  4.21it/s]

🔍 Test Set Evaluation:
Accuracy:  0.7929
Precision: 0.6546
Recall:    0.6946
F1 Score:  0.6740
AUROC:     0.8461

Confusion Matrix:
[[4489  876]
 [ 730 1660]]

Detailed Classification Report:
              precision    recall  f1-score   support

           0     0.8601    0.8367    0.8483      5365
           1     0.6546    0.6946    0.6740      2390

    accuracy                         0.7929      7755
   macro avg     0.7574    0.7656    0.7611      7755
weighted avg     0.7968    0.7929    0.7945      7755



In [ ]:
# Function to evaluate student model on validation set

def evaluate_on_validation(model, val_dataset, tokenizer, batch_size=8, device="cuda"):
    model.eval()
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        collate_fn=lambda batch: collate_fn_with_tokenizer(batch, tokenizer)
    )

    all_logits = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = {k: v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)

            outputs = model(**inputs)
            logits = outputs.logits
            all_logits.append(logits.cpu())
            all_labels.append(labels.cpu())

    logits = torch.cat(all_logits)
    labels = torch.cat(all_labels)
    probs = torch.nn.functional.softmax(logits, dim=1)
    preds = torch.argmax(probs, dim=1).numpy()
    labels_np = labels.numpy()

    metrics = {
        "accuracy":  accuracy_score(labels_np, preds),
        "precision": precision_score(labels_np, preds),
        "recall":    recall_score(labels_np, preds),
        "f1":        f1_score(labels_np, preds),
        "auroc":     roc_auc_score(labels_np, probs[:, 1].numpy())
    }

    return metrics

# The above function needs this code
def collate_fn_with_tokenizer(batch, tokenizer):
    texts = [x["clean_relevant_note_truncate"] for x in batch]
    labels = torch.tensor([x["label"] for x in batch])
    encodings = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    return encodings, labels

In [ ]:
# Set a Hugging Face project/repo details
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/Postdoc/results/student/tiny-v2/"
teacher_model.to(device)
student_model.to(device)

# Training loop over 6 epochs
for epoch in range(6):
    student_model.train()
    total_loss = 0
    total_ce_loss = 0
    total_kl_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        teacher_inputs = {k: v.to(device) for k, v in batch['teacher_inputs'].items()}
        student_inputs = {k: v.to(device) for k, v in batch['student_inputs'].items()}
        labels = batch['labels'].to(device)

        # Teacher outputs
        with torch.no_grad():
            teacher_logits = teacher_model(**teacher_inputs).logits

        # Student outputs
        student_logits = student_model(**student_inputs).logits

        # Loss components
        loss_ce = ce_loss_fn(student_logits, labels)
        loss_kl = kl_loss_fn(
            nn.functional.log_softmax(student_logits / temperature, dim=-1),
            nn.functional.softmax(teacher_logits / temperature, dim=-1)) * (temperature ** 2)

        # Total loss
        loss = alpha * loss_ce + (1 - alpha) * loss_kl
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
        total_ce_loss += loss_ce.item()
        total_kl_loss += loss_kl.item()

    avg_loss = total_loss / len(train_loader)
    avg_ce_loss = total_ce_loss / len(train_loader)
    avg_kl_loss = total_kl_loss / len(train_loader)

    print(f"\nEpoch {epoch+1} loss: {avg_loss:.4f} | CE loss: {avg_ce_loss:.4f} | KL loss: {avg_kl_loss:.4f}")

    val_metrics = evaluate_on_validation(student_model, val_dataset, student_tokenizer, device=device)
    print(f"Validation metrics after epoch {epoch+1}:")
    for metric, value in val_metrics.items():
        print(f"{metric.capitalize():>9}: {value:.4f}")

    # Save checkpoint
    epoch_ckpt_dir = os.path.join(PROJECT_DIR, f"checkpoint-epoch{epoch+1}")
    student_model.save_pretrained(epoch_ckpt_dir)
    student_tokenizer.save_pretrained(epoch_ckpt_dir)


Epoch 1: 100%|██████████| 4524/4524 [19:06<00:00,  3.95it/s]



Epoch 1 loss: 0.4347 | CE loss: 0.5656 | KL loss: 0.3037
Validation metrics after epoch 1:
 Accuracy: 0.7866
Precision: 0.6631
   Recall: 0.6274
       F1: 0.6448
    Auroc: 0.8182


Epoch 2: 100%|██████████| 4524/4524 [19:03<00:00,  3.95it/s]



Epoch 2 loss: 0.3425 | CE loss: 0.5104 | KL loss: 0.1746
Validation metrics after epoch 2:
 Accuracy: 0.7958
Precision: 0.6915
   Recall: 0.6115
       F1: 0.6491
    Auroc: 0.8281


Epoch 3: 100%|██████████| 4524/4524 [19:11<00:00,  3.93it/s]


...


Epoch 4 loss: 0.2908 | CE loss: 0.4617 | KL loss: 0.1199
Validation metrics after epoch 4:
 Accuracy: 0.8013
Precision: 0.7007
   Recall: 0.6220
       F1: 0.6590
    Auroc: 0.8362


Epoch 5: 100%|██████████| 4524/4524 [19:07<00:00,  3.94it/s]



Epoch 5 loss: 0.2730 | CE loss: 0.4389 | KL loss: 0.1071
Validation metrics after epoch 5:
 Accuracy: 0.7964
Precision: 0.6858
   Recall: 0.6282
       F1: 0.6558
    Auroc: 0.8368


Epoch 6: 100%|██████████| 4524/4524 [19:03<00:00,  3.96it/s]



Epoch 6 loss: 0.2594 | CE loss: 0.4200 | KL loss: 0.0989
Validation metrics after epoch 6:
 Accuracy: 0.7657
Precision: 0.6015
   Recall: 0.7143
       F1: 0.6530
    Auroc: 0.8374


In [ ]:
# Save student model
student_model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model/v2/epoch6")
student_tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model/v2/epoch6")

('/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model/v2/epoch6/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model/v2/epoch6/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model/v2/epoch6/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model/v2/epoch6/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Postdoc/BERT-tiny-student-model/v2/epoch6/tokenizer.json')

In [ ]:
# Trained student model performance
print("Trained student model:")
results = evaluate_model_on_test_set_raw(student_model, test_dataset, tokenizer_name="gaunernst/bert-tiny-uncased")

Trained student model:


Evaluating: 100%|██████████| 970/970 [00:15<00:00, 63.57it/s]

🔍 Test Set Evaluation:
Accuracy:  0.7810
Precision: 0.6242
Recall:    0.7276
F1 Score:  0.6719
AUROC:     0.8403

Confusion Matrix:
[[4318 1047]
 [ 651 1739]]

Detailed Classification Report:
              precision    recall  f1-score   support

           0     0.8690    0.8048    0.8357      5365
           1     0.6242    0.7276    0.6719      2390

    accuracy                         0.7810      7755
   macro avg     0.7466    0.7662    0.7538      7755
weighted avg     0.7935    0.7810    0.7852      7755



In [ ]:
from sklearn.metrics import precision_recall_fscore_support

# Trying threshold optimization (optional)

def collate_fn_with_tokenizer(batch, tokenizer, device="cuda"):
    texts = [x["clean_relevant_note_truncate"] for x in batch]
    labels = torch.tensor([x["label"] for x in batch])
    encodings = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    return {k: v.to(device) for k, v in encodings.items()}, labels.to(device)

def threshold_optimization_raw(model, test_dataset, tokenizer, device="cuda", batch_size=8):
    model.eval()
    model.to(device)

    loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=lambda batch: collate_fn_with_tokenizer(batch, tokenizer, device)
    )

    all_probs = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            logits = model(**inputs).logits
            probs = torch.softmax(logits, dim=-1)[:, 1]
            all_probs.append(probs.cpu())
            all_labels.append(labels.cpu())

    all_probs = torch.cat(all_probs).numpy()
    all_labels = torch.cat(all_labels).numpy()

    # Threshold sweep
    best_f1 = 0
    best_thresh = 0.5

    for thresh in np.arange(0.1, 0.9, 0.01):
        preds_thresh = (all_probs >= thresh).astype(int)
        _, _, f1, _ = precision_recall_fscore_support(
            all_labels, preds_thresh, average="binary", zero_division=0
        )
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh

    print(f"Best F1: {best_f1:.4f} at threshold: {best_thresh:.2f}")
    return best_thresh, best_f1

In [ ]:
best_thresh, best_f1 = threshold_optimization_raw(
    model=student_model,
    test_dataset=test_dataset,
    tokenizer=student_tokenizer,
    device="cuda")

Best F1: 0.6764 at threshold: 0.62
